### O que iremos aprender

* 01 - Importando Bibliotecas
* 02 - Criando uma SparkSession
* 03 - Configurando o Modo de Exibição de um DataFrame
* 04 - Arquivo Para Ser Importado (Arquivo de Seleções)
* 05 - Leitura do Arquivo, Criação de um DataFrame e Ordenando
* 06 - Definindo a Estrutura do DataFrame, Leitura do Arquivo e Ordenando
* 07 - Criando um TempView (Modelo de Tabela Temporária para ser Manipulada com Linguagem SQL)
* 08 - Leitura do DataFrame com Linguagem SQL e Ordenando o Resultado
* 09 - Leitura do DataFrame com Linguagem SQL e Ordenando o Resultado
* 10 - Leitura do DataFrame com Linguagem SQL e Ordenando (sem informa o nome da coluna) o Resultado
* 11 - Arquivo Para Ser Importado (Arquivo de Jogadores)

### Os arquivos usados estão disponíveis no GitHub

https://github.com/dbaassists/ModelagemDados/tree/main/BaseDados_Copa_Catar_2022

# 01 - Importando Bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark import SparkFiles

# 02 - Criando uma SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

# 03 - Configurando o Modo de Exibição de um DataFrame

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", "true")

# 04 - Arquivo Para Ser Importado (Arquivo de Seleções)

In [4]:
arquivoSelecao = '../BaseDados_Copa_Catar_2022/catar2022_base_selecoes.csv'

# 05 - Leitura do Arquivo, Criação de um DataFrame e Ordenando

In [5]:
dfSelecao = spark.read.csv(arquivoSelecao
                            ,header=True
                            ,sep=';'
                            ,encoding='latin1')

dfSelecao.orderBy(F.col('NOMSELECAO').desc()).show(2,truncate=False)


+----------+----------------+------------+
|NOMSELECAO|DSCORIGEMSELECAO|NOMTREINADOR|
+----------+----------------+------------+
|URUGUAI   |SA              |DIEGO ALONSO|
|TUNÍSIA   |AF              |JALEL KADRI |
+----------+----------------+------------+
only showing top 2 rows



# 06 - Definindo a Estrutura do DataFrame, Leitura do Arquivo e Ordenando

In [6]:

schema = StructType() \
      .add("NomSelecao",StringType(),True) \
      .add("DscOrigemSelecao",StringType(),True) \
      .add("NomTreinador",StringType(),True)


dfSelecao = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoSelecao)


dfSelecao.orderBy(F.col('NOMSELECAO').desc()).show(2,truncate=False)

+----------+----------------+------------+
|NomSelecao|DscOrigemSelecao|NomTreinador|
+----------+----------------+------------+
|URUGUAI   |SA              |DIEGO ALONSO|
|TUNÍSIA   |AF              |JALEL KADRI |
+----------+----------------+------------+
only showing top 2 rows



# 07 - Criando um TempView (Modelo de Tabela Temporária para ser Manipulada com Linguagem SQL)

In [7]:
dfSelecao.createOrReplaceTempView('selecao')

# 08 - Leitura do DataFrame com Linguagem SQL e Ordenando o Resultado

Usando "as" para renomear colunas

In [8]:
spark.sql("""SELECT NomSelecao as NomeSelecao
             ,DscOrigemSelecao as Continente
             ,NomTreinador as Treinador
             FROM selecao
             ORDER BY NomSelecao DESC""").show(2, truncate=False)

+-----------+----------+------------+
|NomeSelecao|Continente|Treinador   |
+-----------+----------+------------+
|URUGUAI    |SA        |DIEGO ALONSO|
|TUNÍSIA    |AF        |JALEL KADRI |
+-----------+----------+------------+
only showing top 2 rows



# 09 - Leitura do DataFrame com Linguagem SQL e Ordenando o Resultado

Não usando "as" para renomear colunas

In [9]:
spark.sql("""SELECT NomSelecao NomeSelecao
             ,DscOrigemSelecao Continente
             ,NomTreinador Treinador
             FROM selecao
             ORDER BY NomSelecao DESC""").show(2, truncate=False)

+-----------+----------+------------+
|NomeSelecao|Continente|Treinador   |
+-----------+----------+------------+
|URUGUAI    |SA        |DIEGO ALONSO|
|TUNÍSIA    |AF        |JALEL KADRI |
+-----------+----------+------------+
only showing top 2 rows



# 10 - Leitura do DataFrame com Linguagem SQL e Ordenando (sem informa o nome da coluna) o Resultado

In [10]:
spark.sql("""SELECT NomSelecao as NomeSelecao
             ,DscOrigemSelecao as Continente
             ,NomTreinador as Treinador
             FROM selecao
             ORDER BY 1 DESC""").show(2, truncate=False)

+-----------+----------+------------+
|NomeSelecao|Continente|Treinador   |
+-----------+----------+------------+
|URUGUAI    |SA        |DIEGO ALONSO|
|TUNÍSIA    |AF        |JALEL KADRI |
+-----------+----------+------------+
only showing top 2 rows



# 11 - Arquivo Para Ser Importado (Arquivo de Jogadores)

In [11]:
arquivoJogadores = '../BaseDados_Copa_Catar_2022/catar2022_base_jogadores.csv'

In [12]:
dfJogadores = spark.read.csv(arquivoJogadores
                            ,sep=';'
                            ,header=True
                            ,encoding='latin1')

dfJogadores.show(2)

+------------+---------+----------+----------+
|  NOMJOGADOR|NUMCAMISA|NOMPOSICAO|NOMSELECAO|
+------------+---------+----------+----------+
|    AL-SHEEB|        1|       GOL|     CATAR|
|PEDRO MIGUEL|        2|       LAD|     CATAR|
+------------+---------+----------+----------+
only showing top 2 rows



# 12 - Relacionamento entre DataFrames

In [13]:
dfSelecao.join(dfJogadores
               ,on=['NOMSELECAO','NOMSELECAO']
               ,how='inner')\
                .select(dfSelecao['NomSelecao']
                       ,dfSelecao['DscOrigemSelecao']
                       ,dfJogadores['NOMJOGADOR']
                       ,dfJogadores['NUMCAMISA']
                       ,dfJogadores['NOMPOSICAO']
                       ,dfSelecao['NomTreinador']).show(2)

+----------+----------------+------------+---------+----------+-------------+
|NomSelecao|DscOrigemSelecao|  NOMJOGADOR|NUMCAMISA|NOMPOSICAO| NomTreinador|
+----------+----------------+------------+---------+----------+-------------+
|     CATAR|              AS|    AL-SHEEB|        1|       GOL|FÉLIX SÁNCHEZ|
|     CATAR|              AS|PEDRO MIGUEL|        2|       LAD|FÉLIX SÁNCHEZ|
+----------+----------------+------------+---------+----------+-------------+
only showing top 2 rows



In [14]:
dfSelecao.join(dfJogadores
               ,on=['NOMSELECAO','NOMSELECAO']
               ,how='inner')\
                .select('DscOrigemSelecao').show(2)

+----------------+
|DscOrigemSelecao|
+----------------+
|              AS|
|              AS|
+----------------+
only showing top 2 rows



In [15]:
dfSelecao.join(dfJogadores
               ,on=['NOMSELECAO','NOMSELECAO']
               ,how='inner')\
                .select(F.col('DscOrigemSelecao')).show(2)

+----------------+
|DscOrigemSelecao|
+----------------+
|              AS|
|              AS|
+----------------+
only showing top 2 rows



In [16]:
dfSelecao.join(dfJogadores
               ,on=['NOMSELECAO','NOMSELECAO']
               ,how='inner')\
                .select(dfSelecao['DscOrigemSelecao']).show(2)

+----------------+
|DscOrigemSelecao|
+----------------+
|              AS|
|              AS|
+----------------+
only showing top 2 rows



In [17]:
dfJogadores.createOrReplaceTempView('jogador')

In [18]:
import pandas as pd

dfSelecaoJogador = spark.sql("""
SELECT S.NomSelecao
,S.DscOrigemSelecao
,J.NOMJOGADOR
,J.NUMCAMISA
,J.NOMPOSICAO
,S.NomTreinador 
FROM selecao S

INNER JOIN jogador J

ON S.NOMSELECAO = J.NOMSELECAO

""")

dfPandasSelecaoJogador = dfSelecaoJogador.toPandas()

dfPandasSelecaoJogador.to_csv('../ProjetoSparkKB/Dados/SelecaoJogador.csv', sep=';', index=False, encoding='latin1')

In [19]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
spark.sql("create database dbaassists")

""


In [21]:
spark.sql("show databases").show()

+----------+
| namespace|
+----------+
|dbaassists|
|   default|
+----------+



In [22]:
spark.sql("use dbaassists")

""


In [23]:
spark.sql("use default")

""


In [24]:
spark.sql("""DROP TABLE IF EXISTS dbaassists.SelecaoJogador""")

""


In [25]:
dfSelecaoJogador.write.mode('overwrite').saveAsTable('dbaassists.SelecaoJogador')

AnalysisException: Can not create the managed table('`dbaassists`.`SelecaoJogador`'). The associated location('file:/C:/Temp/ProjetoSparkKB/spark-warehouse/dbaassists.db/selecaojogador') already exists.

In [ ]:
spark.sql("show tables")

In [ ]:
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
dfteste = spark.createDataFrame(data=data2,schema=schema)
dfteste.printSchema()
dfteste.show()

In [ ]:
spark.sql('SELECT * FROM dbaassists.SelecaoJogador')

In [ ]:
spark.sql('desc dbaassists.SelecaoJogador')

In [ ]:
spark.sql('''
SELECT NOMPOSICAO , COUNT(1) TOTAL
FROM dbaassists.SelecaoJogador
GROUP BY NOMPOSICAO 
ORDER BY 2 DESC
''')

In [ ]:
spark.sql('''
DELETE
FROM dbaassists.SelecaoJogador
WHERE NOMPOSICAO = 'ATA'
''')